In [1]:
import pandas as pd
import numpy as np

In [2]:
#The difference between this and the first version is many. 1) I would like to do the score on only p53 WT cell lines
#2) the pevious version only counted cells whos expression was above the global average for each key gene
#however is might be more useful to have a more stringent test perhaps 1 sigma? 2 sigma? 
#there is also the matter is literally counting mutations in the cBAF complex and mutations that are noted as enriched in
#cBAFloss like cell lines. 
# in any case, I will try these one at a time and then test it against the TCGA database to hopefully replicate the findings
#of the mol cell paper. DOI:https://doi.org/10.1016/j.molcel.2023.03.013

In [3]:
Omics_EPCG_TPMlogp1 = pd.read_csv('OmicsExpressionProteinCodingGenesTPMLogp1.csv')
Omics_EPCG_TPMlogp1 = Omics_EPCG_TPMlogp1.rename(columns={'Unnamed: 0': 'Depmap ID'})
Omics_EPCG_TPMlogp1.columns = Omics_EPCG_TPMlogp1.columns.str.split(' ').str[0]
Omics_EPCG_TPMlogp1.head()

,Depmap,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1
0,ACH-001113,4.331992,0.000000,7.364660,2.792855,4.471187,0.028569,1.226509,3.044394,6.500005,...,2.689299,0.189034,0.201634,2.130931,0.555816,0.0,0.275007,0.0,0.0,0.000000
1,ACH-001289,4.567424,0.584963,7.106641,2.543496,3.504620,0.000000,0.189034,3.813525,4.221877,...,1.286881,1.049631,0.321928,1.464668,0.632268,0.0,0.014355,0.0,0.0,0.000000
2,ACH-001339,3.150560,0.000000,7.379118,2.333424,4.228049,0.056584,1.310340,6.687201,3.682573,...,0.594549,1.097611,0.831877,2.946731,0.475085,0.0,0.084064,0.0,0.0,0.042644
3,ACH-001538,5.085340,0.000000,7.154211,2.545968,3.084064,0.000000,5.868390,6.165309,4.489928,...,0.214125,0.632268,0.298658,1.641546,0.443607,0.0,0.028569,0.0,0.0,0.000000
4,ACH-000242,6.729417,0.000000,6.537917,2.456806,3.867896,0.799087,7.208478,5.570159,7.127117,...,1.117695,2.358959,0.084064,1.910733,0.000000,0.0,0.464668,0.0,0.0,0.000000


In [8]:
#def calculate_average (df):
 #   avg_values = {}
    
    # Loop through each column in the DataFrame
  #  for column in df.columns:
        # Check if the column contains numeric data
   #     if pd.api.types.is_numeric_dtype(df[column]):
            # Check for missing values in target_data and column
    #        if df[column].isnull().any():
       #         avg_values[column] = np.nan
     #       else:
                # Check if the column has more than one unique value
      #          if df[column].nunique() > 1:
                    # Calculate correlation with the target column
        #            average = np.mean(df[column])
         #           avg_values[column] = average
          #      else:
           #         avg_values[column] = np.nan
    
   # return avg_values

def calculate_25th_percentile(df):
    percentile_values = {}
    
    # Loop through each column in the DataFrame
    for column in df.columns:
        # Check if the column contains numeric data
        if pd.api.types.is_numeric_dtype(df[column]):
            # Filter out NaN and zero values
            non_zero_values = df[column][(df[column] != 0) & (df[column].notnull())]
            
            # Check if there are remaining values after filtering
            if len(non_zero_values) > 0:
                # Check if the column has more than one unique non-zero value
                if non_zero_values.nunique() > 1:
                    # Calculate the 75th percentile of the non-zero values
                    percentile_25 = np.percentile(non_zero_values, 25)
                    percentile_values[column] = percentile_25
                else:
                    percentile_values[column] = np.nan
            else:
                percentile_values[column] = np.nan
    
    return percentile_values


In [ ]:
def calculate_average(df):
    avg_values = {}
    
    # Loop through each column in the DataFrame
    for column in df.columns:
        # Check if the column contains numeric data
        if pd.api.types.is_numeric_dtype(df[column]):
            # Filter out zeros and NaN values
            non_zero_values = df[column][(df[column] != 0) & (df[column].notnull())]
            
            # Check if there are remaining values after filtering
            if len(non_zero_values) > 0:
                # Check if the column has more than one unique non-zero value
                if non_zero_values.nunique() > 1:
                    # Calculate the average of the non-zero values
                    average = np.mean(non_zero_values)
                    avg_values[column] = average
                else:
                    avg_values[column] = np.nan
            else:
                avg_values[column] = np.nan
    
    return avg_values


In [ ]:
def calculate_std (df):
    std_values = {}
    
    # Loop through each column in the DataFrame
    for column in df.columns:
        # Check if the column contains numeric data
        if pd.api.types.is_numeric_dtype(df[column]):
            # Check for missing values in the column
            if df[column].isnull().any():
                std_values[column] = np.nan
            else:
                # Check if the column has more than one unique value
                if df[column].nunique() > 1:
                    # Calculate the standard deviation of the column
                    std_dev = np.std(df[column])
                    std_values[column] = std_dev
                else:
                    std_values[column] = np.nan
    
    return std_values

In [ ]:
RNA_avg = calculate_average(Omics_EPCG_TPMlogp1)

In [ ]:
print(RNA_avg)

In [9]:
RNA_per = calculate_25th_percentile(Omics_EPCG_TPMlogp1)

In [ ]:
print(RNA_per)

In [10]:
reference = pd.DataFrame(data=RNA_per, index=[0])
reference = reference.rename(index={0:'Average'})
reference = reference.transpose()
reference = reference.reset_index()
reference = reference.rename(columns={'index':'gene'})

In [11]:
reference.head()

,gene,Average
0,TSPAN6,3.002702
1,TNMD,0.028569
2,DPM1,6.113846
3,SCYL3,2.007196
4,C1orf112,3.226508


In [12]:
ref2 = Omics_EPCG_TPMlogp1.transpose()
ref2 = ref2.reset_index()


row_index = 0
# Set the new header
new_header = ref2.iloc[row_index]  # Take the row to be used as header
df = ref2[1:]  # Remove the header row from the data
df.columns = new_header  # Set new header

# Reset index if desired
df.reset_index(drop=True, inplace=True)
df = df.rename(columns={'Depmap':'gene'})

In [13]:
df.head()

,gene,ACH-001113,ACH-001289,ACH-001339,ACH-001538,ACH-000242,ACH-000708,ACH-000327,ACH-000233,ACH-000461,...,ACH-001578,ACH-000036,ACH-000973,ACH-001128,ACH-000750,ACH-000285,ACH-002669,ACH-001858,ACH-001997,ACH-000052
0,TSPAN6,4.331992,4.567424,3.15056,5.08534,6.729417,4.27277,3.337711,0.056584,4.017031,...,6.345361,4.018812,4.328406,5.995032,3.533563,0.056584,3.111031,4.390943,5.05745,4.249445
1,TNMD,0.0,0.584963,0.0,0.0,0.0,0.189034,0.0,0.0,0.0,...,3.401903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DPM1,7.36466,7.106641,7.379118,7.154211,6.537917,7.023255,5.927659,6.094236,6.534497,...,7.211694,6.700856,7.059182,6.238978,6.488483,6.604368,7.031329,7.013239,7.815191,6.175724
3,SCYL3,2.792855,2.543496,2.333424,2.545968,2.456806,2.555816,1.944858,3.971773,2.226509,...,2.533563,2.137504,1.891419,2.304511,1.823749,3.266037,1.541019,1.887525,2.538538,2.31904
4,C1orf112,4.471187,3.50462,4.228049,3.084064,3.867896,3.841973,2.678072,3.731183,3.02148,...,4.374344,2.531069,3.529821,4.0,3.308885,4.973152,3.664483,3.252476,3.893362,3.825786


In [14]:
cBAF_downreg = pd.read_excel('cBAF downreg.xlsx')

In [15]:
cBAF_score = pd.merge(df, reference, on = 'gene', how = 'inner')
cBAF_score = pd.merge(cBAF_score, cBAF_downreg, on = 'gene', how = 'inner')

In [16]:
cBAF_score.head()

,gene,ACH-001113,ACH-001289,ACH-001339,ACH-001538,ACH-000242,ACH-000708,ACH-000327,ACH-000233,ACH-000461,...,proportion_of_tissues_where_significant,TCGA-COAD_qvalue,TCGA-LUAD_qvalue,TCGA-LUSC_qvalue,TCGA-BRCA_qvalue,TCGA-KIRP_qvalue,TCGA-READ_qvalue,TCGA-LIHC_qvalue,TCGA-BLCA_qvalue,annotated_functions
0,STAB1,0.056584,0.084064,0.070389,0.0,0.014355,0.137504,0.028569,0.042644,0.201634,...,0.5,0.622912,0.928117,0.999958,0.019496,3.577760e-08,0.937959,0.018408,0.003142,protein binding
1,SLC6A7,0.0,0.0,0.0,0.042644,0.0,1.970854,0.014355,0.0,0.0,...,0.5,0.918358,0.015666,0.999958,0.032801,3.664958e-04,0.802462,0.029678,0.488808,None
2,ALOX5,0.214125,0.214125,0.014355,0.62293,5.782933,1.944858,0.201634,0.238787,1.077243,...,0.5,0.678948,0.043900,0.999958,0.016656,9.481017e-03,0.569912,0.000022,0.067270,protein binding
3,MTMR11,1.831877,4.831877,1.669027,3.778209,1.765535,3.91265,2.759156,2.970854,2.865919,...,0.5,0.613535,0.878440,0.999958,0.008080,5.340482e-01,0.006816,0.002723,0.000059,None
4,ADAM28,0.111031,0.097611,0.124328,2.811471,0.978196,0.0,0.214125,1.819668,0.176323,...,0.5,0.049856,0.700369,0.999958,0.016491,3.649882e-02,0.992190,0.000002,0.124470,None


In [17]:
def check_numbers(row):
    # 'Average' is the reference number for comparison
    defined_number = row['Average']
    
    # Initialize results dictionary
    results = {}
    
    # Iterate through each column except 'Average'
    for col in row.index:
        if col != 'Average':
            value = row[col]
            
            # Try to convert value to a float
            try:
                value = float(value)
                # Check if value is less than defined_number and not zero
                results[col] = (value < defined_number and value != 0)
            except ValueError:
                # If conversion fails, skip this value
                results[col] = False
    
    return results

In [18]:
# Apply the function to each row and store the results in a new column
cBAF_score['CheckResults'] = cBAF_score.apply(check_numbers, axis=1)

In [19]:
# Initialize a dictionary to hold the count of True values for each key
true_counts = {}

# Iterate over each row in the 'CheckResults' column
for results in cBAF_score['CheckResults']:
    for key, value in results.items():
        if value:  # If the value is True
            if key in true_counts:
                true_counts[key] += 1
            else:
                true_counts[key] = 1


In [20]:
counts = pd.DataFrame(list(true_counts.items()), columns=['Depmap ID','Count'])

In [21]:
counts.head()

,Depmap ID,Count
0,ACH-000242,29
1,ACH-000327,34
2,ACH-000233,18
3,ACH-002040,22
4,ACH-000769,25


In [22]:
DRC_table_all = pd.read_excel('DRCtableAUC_merge_DepmapID.xlsx')
DRC_table_all.head()

,Cell name,Depmap ID,"AUC, DEG-35","AUC, DEG-35 sigfig 4"
0,143B,ACH-001001,0.738025,0.7380
1,22RV1,ACH-000956,0.543824,0.5438
2,2313287,ACH-000948,0.362258,0.3623
3,253J,ACH-000011,0.811752,0.8118
4,253JBV,ACH-000026,0.713113,0.7131


In [23]:
cBAF_score_all = pd.merge(counts, DRC_table_all, on = 'Depmap ID', how = 'inner')

In [24]:
cBAF_score_all.to_excel('cBAF_loss_score_v6.xlsx')

In [25]:
DRC_table_p53wt = pd.read_excel('TP53WT_cells_AUC_DepmapID.xlsx')

In [26]:
DRC_table_p53wt.head()

,Cell name,Depmap ID,"AUC, DEG-35","AUC, DEG-35 sigfig 4"
0,2313287,ACH-000948,0.362258,0.3623
1,253J,ACH-000011,0.811752,0.8118
2,253JBV,ACH-000026,0.713113,0.7131
3,697,ACH-000070,0.549193,0.5492
4,769P,ACH-000411,0.539455,0.5395


In [28]:
cBAF_score_wt = pd.merge(counts,DRC_table_p53wt, on = 'Depmap ID', how = 'inner')
cBAF_score_wt.to_excel('cBAF_loss_score_v7.xlsx')